<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Графики.-Уточнить-легенду!" data-toc-modified-id="Графики.-Уточнить-легенду!-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Графики. Уточнить легенду!</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import bokeh
from bokeh.plotting import figure, show, save, output_file
from bokeh.models import ColumnDataSource
from bokeh.io import export_png
from bokeh.models import Legend,LegendItem
import math
from bokeh.models import Label
from tqdm.notebook import tqdm as tqdm
import time

import sys
sys.path.insert(0, '../') 
from btb_matrix.domain import FullBtBways
from btb_matrix.config import engine

In [ ]:
scenario = 199
_fullbtb = FullBtBways('2021-11-24', scenario, start_time = '7:30')

In [ ]:
matrix_current = _fullbtb.get_trip_flows()

# Добавляев название о.п.
stops = pd.read_sql('select stop_id, stop_name from routes.stops()', con=engine)
stops = stops.set_index('stop_id')
matrix_current = matrix_current.join(stops, on='stop_id_start')
matrix_current = matrix_current.join(stops, on='stop_id_end', lsuffix='_start', rsuffix='_end')


# Добавляем уровень загрузки
matrix_current['rel_inside'] = matrix_current['cnt_inside'] / matrix_current['capacity']

matrix_current.head(3)

In [ ]:
# matrix_current['cnt_inside'] = matrix_current['cnt_inside'] / 2
# matrix_current['cnt_on'] = matrix_current['cnt_on'] / 2
# matrix_current['cnt_off'] = matrix_current['cnt_off'] / 2

# Добавляем уровень загрузки
# matrix_current['rel_inside'] = matrix_current['cnt_inside'] / matrix_current['capacity']

# matrix_current.head(3)

In [ ]:
# matrix_current.to_excel(r'C:\Users\ovnanyan_gm\btb_matrix\notebooks\data\Графики загрузки\177\matrix_current.xlsx', index=False) 

In [ ]:
# файл с остановками МЦД/БКЛ, чтобы отметить на графике
stops_metro = pd.read_excel('stops0.xlsx',  engine='openpyxl', encoding = 'cp1251')
stops_metro = list(stops_metro['stop_id'])
stops_metro[:3]

In [ ]:
# приводим в нужный вид

matrix_current_inputs = matrix_current[['mvn','route_id','segment_seq','stop_id_start','cnt_on','capacity','num_of_trips', 'cnt_inside']]
matrix_current_inputs = matrix_current_inputs.rename(columns={'segment_seq':'stop_sequence',
                                'stop_id_start':'stop_id'})


matrix_current_outputs = matrix_current[['mvn','route_id','stop_id_end','cnt_off']]
matrix_current_outputs = matrix_current_outputs.rename(columns={'stop_id_end':'stop_id'})


mtrx = pd.merge(matrix_current_inputs, matrix_current_outputs, how='outer',on=['mvn','route_id','stop_id'])
mtrx = mtrx.sort_values(['mvn','route_id','stop_sequence','stop_id']).reset_index(drop=True)
mtrx = mtrx.fillna(0)

matrix_current = mtrx.copy()
matrix_current = pd.merge(matrix_current, stops, how='left', on = ['stop_id'])

matrix_current.head(5)  

In [ ]:
# МВН для графика
mvns = [
'КМ-471-00-А-прямое',
'КМ-471-00-А-обратное',
'КМ-472-00-А-прямое',
'КМ-472-00-А-обратное',
'КМ-473-00-А-прямое',
'КМ-473-00-А-обратное',
'КМ-474-00-А-прямое',
'КМ-474-00-А-обратное',
'КМ-475-00-А-прямое',
'КМ-475-00-А-обратное',
'КМ-476-00-А-прямое',
'КМ-476-00-А-обратное'
]

# выгрузка всех графиков
# mvns = list(matrix_current['mvn'].unique())

##### Графики. Уточнить легенду!

In [ ]:
for i in tqdm(mvns):
    
    print(i)
    
    try:
    
        #--------------считаем наполнение сущпол-------------------

        # выбирем данные для нужного МВН
        t = matrix_current[matrix_current['mvn'] == i].copy()
        t = t.reset_index(drop=True)
        freq = t.iloc[0]['num_of_trips']

        #------------записываем значение вместимости----------------------

        capacity = t['capacity'].values[0]

        t['Провозная способность'] = capacity
        t['idx'] = t.index

        #-----------собираем данные для графика---------------------

        cds_input = dict(x=list(t['stop_sequence'].values), # данные о входе
                top=list(t['cnt_on'].values/freq),
               x_offset = [x-0.2 for x in range(len(t))])
        source_input = ColumnDataSource(data = cds_input)

        cds_output = dict(x=list(t['stop_sequence'].values), # данные о выходе
               top=list(t['cnt_off'].values/freq),
               x_offset = [x+0.2 for x in range(len(t))])
        source_output = ColumnDataSource(data = cds_output) 


        cds_volume = dict(x=list(t['idx'].values), # данные о наполнении
               y=list(t['cnt_inside'].values/freq),
               cap=list(t['Провозная способность'].values/freq),
                         )
        source_volume = ColumnDataSource(data = cds_volume)

        # ----------станции БКЛ/МЦД------------------------

        metro = list(t[t['stop_id'].isin(stops_metro)].index) # станции метро, которые надо отметить
        print(metro)

        #------------рисуем график----------------------

        p = figure(plot_width=1200, plot_height=800)

        r0 = p.vbar(x='x_offset', width=0.4, bottom=0,
           top='top', color="orange", source = source_input, )

        r1 = p.vbar(x='x_offset', width=0.4, bottom=0,
           top='top', color="royalblue", source = source_output, )

        r4 = p.line(x='x', y='y', line_width=2, color="blue", source = source_volume, )

        p.circle(x='x', y='y', size=8, color="blue", source = source_volume)

        r5 = p.line(x='x', y='cap', line_width=2, color="red", source = source_volume, )


        if metro:
            for m in metro:
                r7 = p.line(x=[m]*2, y = [0,0.75*capacity/freq], color = 'black',line_dash='dotted', line_width=2,)


        else:
            r7 = p.line(x=0, y=0, color='black',line_dash='dotted',line_width=2,)

        #-------------легенда------------------------

        legend = Legend(items=[
            ("Посадка"   , [r0]),
            ("Высадка" , [r1]),
            ("Наполнение" , [r4]),
            ("Провозная способность" , [r5]),
        #   ("Новая станция метро" , [r7])
        ], location="center")

        p.add_layout(legend, 'right')

        #-------------оси---------------------------

        ax_list = list(t.index)
        p.xaxis.ticker = ax_list 

#         p.xaxis.major_label_overrides = (t.set_index('idx')['stop_id']).astype('str').to_dict() #подписи названием
        p.xaxis.major_label_overrides = t.set_index('idx')['stop_name'].to_dict() #подписи названием
        p.xaxis.major_label_orientation = -math.pi/6 #если подписи нужно повернуть

        #-------------- форматирование подписей осей----------------  

        p.xaxis.major_label_text_font_size = '10pt'
        p.xaxis.major_label_text_align = 'center'
        p.xaxis.major_label_text_font_style = 'bold'


        p.yaxis.major_label_text_font_size = '10pt'
        p.yaxis.major_label_text_font_style = 'bold'

        #---------------устанавливаем границы графика (нужно расширить, если подписи не помещаются)

        p.min_border_right = 150
        p.min_border_left = 150

        p.title.text = str(i)
        p.title.align = "center"
        p.title.text_font_size = "25px"

        p.toolbar_location = None

        #-----------сохраняем---------------
        #   output_file(r"C:\Users\ovnanyan_gm\stages_occupancy\data\Графики загрузки\58\{0}.html".format(i))
        #   save(p)
        export_png(p, filename=r"C:\Users\ovnanyan_gm\btb_matrix\notebooks\data\Графики загрузки\%s\{0}.png".format(i) %(scenario))
        print('saved')
        
    except:
        print('error')